<a href="https://colab.research.google.com/github/aritrasen87/LLM/blob/main/LLama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
import os
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [2]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install hf_transfer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from IPython.display import Markdown, display

In [4]:
!nvidia-smi

Sat Apr 20 03:41:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import os
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM , BitsAndBytesConfig

In [6]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [8]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    model_kwargs={"torch_dtype": torch.bfloat16,
                  "quantization_config": {"load_in_4bit" : True},
                  "low_cpu_mem_usage": True},
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {"role": "user", "content": input_text},
    ]

    prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
            )
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipe(
        prompt,
        max_new_tokens=max_length,
        eos_token_id=terminators,
        do_sample=False,
        temperature=0.0,
        top_p=0.9,
    )

    generated_outputs = outputs[0]["generated_text"]
    text = outputs[0]["generated_text"][len(prompt):]
    wrapped_text = wrap_text(text)
    display(Markdown(wrapped_text))

In [10]:
generate('Write a detailed analogy between Yoga and a Spirituality.',
         system_prompt="You are a helpful Spiritual assistant. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=1024)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What a wonderful question! As a Spiritual assistant, I'd be delighted to help you explore
the connection between Yoga and Spirituality. Here's a step-by-step analogy to help you
understand the relationship between these two practices:

**Step 1: Understanding Yoga**

Yoga is a physical, mental, and spiritual practice that originated in ancient India. It
involves various postures (asanas), breathing techniques (pranayama), and meditation to
unite the body, mind, and spirit. Yoga aims to balance the body's energy, calm the mind,
and connect with the inner self.

**Step 2: Understanding Spirituality**

Spirituality is a personal and subjective experience that connects us to a higher power,
the universe, or a deeper sense of purpose. It's a journey to understand our place in the
world, our values, and our values. Spirituality can be practiced through various means,
such as meditation, prayer, or simply being present in nature.

**Step 3: The Connection between Yoga and Spirituality**

Now, let's explore the connection between Yoga and Spirituality. Just as Yoga aims to
unite the body, mind, and spirit, Spirituality seeks to connect us with a higher power or
a deeper sense of purpose. Both practices share a common goal: to find inner peace,
balance, and harmony.

**Analogy: The Body, Mind, and Spirit**

Imagine the body as a physical temple, the mind as a mental sanctuary, and the spirit as a
sacred inner space. Just as Yoga practices help to balance and harmonize the body and
mind, Spirituality practices help to connect and align the body, mind, and spirit.

**Step 4: The Practice of Yoga as a Spiritual Practice**

When we practice Yoga, we're not just stretching our bodies or calming our minds; we're
also cultivating awareness, focus, and inner peace. These qualities are essential for
spiritual growth and connection. As we move through Yoga postures, we're not just
exercising our physical bodies; we're also awakening our inner selves.

**Step 5: The Connection to the Divine**

In Yoga, we often connect with the divine through the practice of meditation, pranayama,
or simply being present in the moment. This connection to the divine is a fundamental
aspect of Spirituality. As we practice Yoga, we're not just connecting with our inner
selves; we're also connecting with the universe, the higher power, or the divine.

**Conclusion**

In conclusion, Yoga and Spirituality are interconnected practices that share a common
goal: to find inner peace, balance, and harmony. Yoga practices help to balance the body
and mind, while Spirituality practices help to connect us with a higher power or a deeper
sense of purpose. By combining these practices, we can cultivate a deeper understanding of
ourselves and the world around us.

I hope this analogy has helped you understand the connection between Yoga and
Spirituality!

In [11]:
generate('Write a detailed history of LLM models.',
         system_prompt="You are a helpful assistant. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=1024)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here's a detailed history of Large Language Models (LLMs) and their evolution over the
years:

**Early Years (1950s-1980s)**

The concept of language processing and machine learning dates back to the 1950s. The first
language processing program, called the "Logical Theorist," was developed by Allen Newell
and Herbert Simon in 1956. This program was designed to simulate human problem-solving
abilities by using logical reasoning and symbolic manipulation.

In the 1960s and 1970s, the development of the first artificial intelligence (AI)
languages, such as Lisp and Prolog, laid the foundation for modern language processing.
These languages were designed to enable computers to reason and solve problems using
logical rules and semantic networks.

**Rule-Based Systems (1980s-1990s)**

The 1980s and 1990s saw the development of rule-based systems, which were designed to
mimic human language processing. These systems used hand-coded rules to analyze and
generate language, but they were limited by their rigidity and inability to generalize.

One notable example of a rule-based system is the "Hearsay" system, developed in the 1980s
by researchers at the University of California, Berkeley. Hearsay was designed to analyze
and generate natural language text, but it relied heavily on hand-coded rules and was not
able to learn from experience.

**Statistical Language Processing (1990s-2000s)**

The 1990s and 2000s saw the development of statistical language processing techniques,
which used machine learning algorithms to analyze and generate language. These techniques
were more flexible and able to learn from large datasets, but they were still limited by
their reliance on hand-coded features and lack of contextual understanding.

One notable example of a statistical language processing system is the "IBM Watson"
system, developed in the 1990s by IBM researchers. Watson was designed to analyze and
generate natural language text, but it relied heavily on hand-coded features and was not
able to learn from experience.

**Deep Learning and Neural Networks (2000s-2010s)**

The 2000s and 2010s saw the development of deep learning and neural networks, which
revolutionized the field of language processing. These techniques used large datasets and
complex neural networks to learn patterns and relationships in language, enabling machines
to understand and generate language with unprecedented accuracy.

One notable example of a deep learning-based language model is the "Word2Vec" model,
developed in the 2010s by researchers at Google. Word2Vec used a neural network to learn
word embeddings, which enabled machines to understand the relationships between words and
generate language that was more natural and human-like.

**Large Language Models (2010s-present)**

The 2010s saw the development of large language models (LLMs), which used massive datasets
and complex neural networks to learn patterns and relationships in language. These models
were designed to be more accurate and flexible than earlier language models, and they have
enabled machines to understand and generate language with unprecedented accuracy.

One notable example of an LLM is the "BERT" model, developed in 2018 by researchers at
Google. BERT used a neural network to learn language patterns and relationships, and it
was trained on a massive dataset of text from the internet. BERT has been used to improve
the accuracy of language processing tasks, such as question answering and text
classification.

**Recent Advances (2020s)**

In recent years, LLMs have continued to evolve and improve, with advances in areas such
as:

* **Multitask learning**: LLMs can be trained on multiple tasks simultaneously, enabling
them to learn more generalizable and transferable knowledge.
* **Transfer learning**: LLMs can be fine-tuned on specific tasks or datasets, enabling
them to adapt to new domains and tasks.
* **Explainability**: LLMs can be designed to provide explanations for their predictions
and decisions, enabling humans to understand and trust the output of the model.

Some notable recent advances in LLMs include:

* **RoBERTa**: Developed in 2019, RoBERTa is a variant of the BERT model that uses a
different training approach and has achieved state-of-the-art results on many language
processing tasks.
* **DistilBERT**: Developed in 2020, DistilBERT is a smaller and more efficient version of
the BERT model that has achieved similar results on many language processing tasks.
* **Longformer**: Developed in 2020, Longformer is a variant of the BERT model that is
designed to handle longer sequences of text and has achieved state-of-the-art results on
many language processing tasks.

Overall, the development of LLMs has revolutionized the field of language processing,
enabling machines to understand and generate language with unprecedented accuracy and
flexibility.